In [14]:
%pylab --no-import-all
%matplotlib inline
import seaborn as sns
import sys
import pandas as pd
from ipywidgets import widgets
from IPython.display import Markdown, display, clear_output, Audio
import os
from os.path import join, isdir
from glob import iglob
import json
from pprint import pformat
import utils
import pickle
import io
import base64
from scipy.io import wavfile
import birdsonganalysis as bsa

sys.path.append('../model')

from song_model import SongModel

sns.set_palette('colorblind')

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [15]:
gridpath = '../model/res/prior_dev_grid_170406_153235//'
run_paths = [run_path for run_path in iglob(join(gridpath, '*')) if isdir(run_path)]

gridanalysis = utils.GridAnalyser(run_paths)

In [16]:
with open(join(gridpath, 'desc.md')) as f:
    display(Markdown(f.read()))

# Run description

## Goals

- Test which are the best parameters for having good voiced signal
- Test if padded is a big issue or not
- Test if padded goes fast or not

## What have changed compared to previous runs?

- I have corrected the weird behaviour in the synthesizer.

## Expected Results (Success condition)

- There is a set of parameters that give a song that sounds like a birdsong

<!-- ## Notes -->


In [12]:
vbox = widgets.VBox()
    
slider = widgets.IntSlider(min=0, max=len(run_paths)-1)
slider.observe(lambda x: gridanalysis.show(x['new'], vbox), names='value')
display(slider)
display(vbox)


In [13]:
display(slider)